In [1]:
from dotenv import load_dotenv

load_dotenv("../.env")
import os

os.environ["HF_HOME"] = "/home/t/.cache/huggingface"


from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.core.postprocessor.llm_rerank import LLMRerank
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)
import asyncio

from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings
from llama_index.core.data_structs import Node
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core import get_response_synthesizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core.postprocessor import SentenceTransformerRerank


from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.schema import NodeWithScore, Node
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import (
    OTLPSpanExporter as HTTPSpanExporter,
)
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
import os
from llama_index.core.workflow import Event
from llama_index.core.schema import NodeWithScore, Node

/home/t/atest/llamaindex-agents/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import (
    TokenTextSplitter,
    HierarchicalNodeParser,
    JSONNodeParser,
    SentenceWindowNodeParser,
    SemanticSplitterNodeParser,
    SemanticDoubleMergingSplitterNodeParser,
    LanguageConfig,
    NodeParser,
    HierarchicalNodeParser,
    TextSplitter,
    MarkdownElementNodeParser,
    MetadataAwareTextSplitter,
    LangchainNodeParser,
    UnstructuredElementNodeParser,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [5]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

In [6]:
from llama_index.embeddings.ollama import OllamaEmbedding

In [7]:
embedding_model = GeminiEmbedding()
embedding_model2 = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5", cache_folder=os.environ["HF_HOME"] + "/hub"
)
embedding_model3 = OllamaEmbedding(model_name="all-minilm")

llm = Gemini(model="models/gemini-1.5-pro")
Settings.llm = llm
Settings.embed_model = embedding_model3

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']


In [8]:
async def get_docs(dirname: str):
    return await SimpleDirectoryReader("./data").aload_data()

In [9]:
docs = await get_docs("./data")

In [10]:
pipeline = IngestionPipeline(transformations=[HierarchicalNodeParser.from_defaults()])
# pipeline = IngestionPipeline(transformations=[SemanticSplitterNodeParser(embed_model=embedding_model2), SentenceWindowNodeParser()])
nodes = pipeline.run(documents=docs)

# node_parser = HierarchicalNodeParser.from_defaults(
#     chunk_sizes=[2048, 512, 128]


# )
# nodes= node_parser.get_nodes_from_documents(docs)

In [11]:
len(nodes), len(docs)

(271, 10)

In [12]:
leaf_nodes = get_leaf_nodes(nodes)
len(leaf_nodes)

222

In [13]:
root_nodes = get_root_nodes(nodes)
len(root_nodes)

10

In [14]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI

docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

In [15]:
## Load index into vector index
from llama_index.core import VectorStoreIndex

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/a

In [16]:
retriever = base_index.as_retriever()
out = retriever.retrieve("who are the authors of this paper")

INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"


In [17]:
out

[NodeWithScore(node=TextNode(id_='9836a8d4-b6c5-411d-9d9c-37821514e51e', embedding=None, metadata={'page_label': '9', 'file_name': '1711.11248v3.pdf', 'file_path': '/home/t/atest/llamaindex-agents/data/1711.11248v3.pdf', 'file_type': 'application/pdf', 'file_size': 2395882, 'creation_date': '2024-08-11', 'last_modified_date': '2024-06-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='41ae8b55-a086-499c-a583-16c74ebbc4d4', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '9', 'file_name': '1711.11248v3.pdf', 'file_path': '/home/t/atest/llamaindex-agents/data/1711.11248v3.pdf', 'file_type': 'application/pdf', 'file_size': 2395882, 'creation_date': '2024-08-11', 'last_modified_date': '2024-06-

## Define Retriever¶


In [18]:
from llama_index.core.retrievers import AutoMergingRetriever

In [19]:
base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

In [20]:
retriever.retrieve("block for video")

INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"


[NodeWithScore(node=TextNode(id_='4512edfc-4f9c-44be-afa9-f0987613406f', embedding=None, metadata={'page_label': '4', 'file_name': '1711.11248v3.pdf', 'file_path': '/home/t/atest/llamaindex-agents/data/1711.11248v3.pdf', 'file_type': 'application/pdf', 'file_size': 2395882, 'creation_date': '2024-08-11', 'last_modified_date': '2024-06-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7e87c6eb-601b-40e5-b2d4-d5908c263745', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '4', 'file_name': '1711.11248v3.pdf', 'file_path': '/home/t/atest/llamaindex-agents/data/1711.11248v3.pdf', 'file_type': 'application/pdf', 'file_size': 2395882, 'creation_date': '2024-08-11', 'last_modified_date': '2024-06-

## Entity metadata Extractor


https://docs.llamaindex.ai/en/stable/examples/metadata_extraction/EntityExtractionClimate/


In [21]:
# # # this downloads 712 MB model

# from llama_index.extractors.entity import EntityExtractor
# from llama_index.core.node_parser import SentenceSplitter

# entity_extractor = EntityExtractor(
#     prediction_threshold=0.5,
#     # label_entities=False,  # include the entity label in the metadata (can be erroneous)
#     # device="cpu",  # set to "cuda" if you have a GPU
# )

# node_parser = SentenceSplitter()

# transformations = [node_parser, entity_extractor]

In [22]:
# pipeline = IngestionPipeline(transformations=transformations)
# nods = pipeline.run(documents=docs)

In [23]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.schema import MetadataMode

In [24]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    BaseExtractor,
)
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.node_parser import TokenTextSplitter

In [25]:
from llama_index.llms.ollama import Ollama

In [26]:
ollama_model = Ollama(model="qwen2:0.5b", request_timeout=120)

In [27]:
text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

extractors = [
    # TitleExtractor(nodes=5, llm=ollama_model),
    QuestionsAnsweredExtractor(questions=3, llm=ollama_model),
    # EntityExtractor(prediction_threshold=0.5),
    # SummaryExtractor(summaries=["prev", "self"], llm=llm),
    KeywordExtractor(keywords=10, llm=ollama_model),
]

transformations = [text_splitter] + extractors

In [28]:
pipeline = IngestionPipeline(transformations=transformations)
nods = pipeline.run(documents=docs)

  5%|▍         | 2/44 [00:09<03:05,  4.41s/it]INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
 52%|█████▏    | 23/44 [01:02<00:59,  2.85s/it]INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
 70%|███████   | 31/44 [01:24<00:41,  3.16s/it]INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
 84%|████████▍ | 37/44 [01:42<00:19,  2.78s/it]INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
100%|██████████| 44/44 [02:09<00:00,  2.94s/it]


In [29]:
from pprint import pprint

In [30]:
def print_node(node: Node | NodeWithScore, metadata: bool = True):
    char = 300
    print("id: ", node.id_)
    print("=" * 10, "METADATA", "=" * 10)
    pprint(node.metadata, indent=4)
    print("=" * 10, "TEXT", "=" * 10)
    print(node.text[:char] + f"...({len(node.text[char:])} chars truncated.)")
    print()

In [31]:
print_node(nods[0])

id:  58424e6b-d40d-4a5a-be80-73b960ddd5e8
========== METADATA ==========
{   'creation_date': '2024-08-11',
    'excerpt_keywords': 'AI Agent, Action Recognition, Video Analysis, '
                        'Residual Learning, Multi-scale Convolution, '
                        'Spatiotemporal Convolution, Deep Learning, Image '
                        'Recognition, State-of-the- art, Hand-Crafted Approach',
    'file_name': '1711.11248v3.pdf',
    'file_path': '/home/t/atest/llamaindex-agents/data/1711.11248v3.pdf',
    'file_size': 2395882,
    'file_type': 'application/pdf',
    'last_modified_date': '2024-06-18',
    'page_label': '1',
    'questions_this_excerpt_can_answer': 'What recent advancements in video '
                                         'analysis have been observed in terms '
                                         'of accuracy and performance on '
                                         'action recognition? How has the '
                                         'des

In [32]:
len(docs[0].text), len(nods)

(4702, 44)

In [33]:
docs[1].text

'namics are jointly intertwined, the (2+1)D blocks (with fac-\ntorized spatial and temporal components) are easier to opti-\nmize. Our experiments demonstrate that ResNets adopting\n(2+1)D blocks homogeneously in all layers achieve state-\nof-the-art performance on both Kinetics and Sports-1M.\n2. Related Work\nVideo understanding is one of the core computer vision\nproblems and has been studied for decades. Many research\ncontributions in video understanding have focused on de-\nveloping spatiotemporal features for video analysis. Some\nproposed video representations include spatiotemporal in-\nterest points (STIPs) [21], SIFT- 3D [27], HOG3D [18],\nMotion Boundary Histogram [5], Cuboids [6], and Action-\nBank [26]. These representations are hand-designed and\nuse different feature encoding schemes such as those based\non histograms or pyramids. Among these hand-crafted\nrepresentations, improved Dense Trajectories (iDT) [38] is\nwidely considered the state-of-the-art, thanks to its s

In [34]:
idex = VectorStoreIndex(nodes=nods)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/a

In [35]:
retriever = idex.as_retriever()
out = retriever.retrieve("filters into separate spatial ")
out

INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"


[NodeWithScore(node=TextNode(id_='e12e6a4e-8554-4889-91a8-feab0c1afd46', embedding=None, metadata={'page_label': '3', 'file_name': '1711.11248v3.pdf', 'file_path': '/home/t/atest/llamaindex-agents/data/1711.11248v3.pdf', 'file_type': 'application/pdf', 'file_size': 2395882, 'creation_date': '2024-08-11', 'last_modified_date': '2024-06-18', 'questions_this_excerpt_can_answer': '1. Is there a difference in the size of the input and output for convolutional layers in higher layers compared to lower layers?\n2. What is the purpose of mixed 3D-2D convolutions? Can you explain how they differ from traditional 2D convolutional layers?\n3. In what way does motion modeling (i.e., 3D convo-\nlutions) differ from traditional 2D convolutional layers in terms of complexity and effectiveness in early layers?\n4. What is the relationship between higher-level summaries provided by surrounding context, such as their relevance to specific projects or topics within the field?', 'excerpt_keywords': 'motio

In [36]:
print_node(out[0])

id:  e12e6a4e-8554-4889-91a8-feab0c1afd46
========== METADATA ==========
{   'creation_date': '2024-08-11',
    'excerpt_keywords': 'motion modeling, early layers, late levels of '
                        'semantic abstraction, convolutional layers, mixed '
                        '3D-2D convolutions, 3D convo-\n'
                        'lutions, high dimensions, temporal modeling, context '
                        'awareness, relevance to specific projects or topics '
                        'within the field, 4D and has size Ni×L×Hi×Wi.',
    'file_name': '1711.11248v3.pdf',
    'file_path': '/home/t/atest/llamaindex-agents/data/1711.11248v3.pdf',
    'file_size': 2395882,
    'file_type': 'application/pdf',
    'last_modified_date': '2024-06-18',
    'page_label': '3',
    'questions_this_excerpt_can_answer': '1. Is there a difference in the size '
                                         'of the input and output for '
                                         'convolutional layers 

In [38]:
out[1].text

'outperform 2D ResNets for\nthe same depth when trained and evaluated on large-scale,\nchallenging action recognition benchmarks such as Sports-\n1M [16] and Kinetics [17].\nInspired by these results, we introduce two new forms\nof spatiotemporal convolution that can be viewed as mid-\ndle grounds between the extremes of 2D (spatial convolu-\ntion) and full 3D. The ﬁrst formulation is named mixed con-\nvolution (MC) and consists in employing 3D convolutions\nonly in the early layers of the network, with 2D convolu-\ntions in the top layers. The rationale behind this design is\nthat motion modeling is a low/mid-level operation that can\nbe implemented via 3D convolutions in the early layers of\na network, and spatial reasoning over these mid-level mo-\ntion features (implemented by 2D convolutions in the top\nlayers) leads to accurate action recognition. We show that\nMC ResNets yield roughly a 3-4% gain in clip-level ac-\ncuracy over 2D ResNets of comparable capacity and they\nmatch th